# 1378. Replace employee id with the unique identifier

In [1]:
import pandas as pd

data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

employees_df = spark.createDataFrame(employees)
employees_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/10/31 19:40:14 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/10/31 19:40:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/31 19:40:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/31 19:40:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/31 19:40:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+---+--------+
|id |name    |
+---+--------+
|1  |Alice   |
|7  |Bob     |
|11 |Meir    |
|90 |Winston |
|3  |Jonathan|
+---+--------+


In [4]:
employee_uni_df = spark.createDataFrame(employee_uni)
employee_uni_df.show(truncate=False)

+---+---------+
|id |unique_id|
+---+---------+
|3  |1        |
|11 |2        |
|90 |3        |
+---+---------+


In [11]:
# in spark Dafaframe

employees_df.join(employee_uni_df,on=['id'],how='left').select('unique_id','name').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+


In [12]:
# in spark SQL

employees_df.createOrReplaceTempView('employees')
employee_uni_df.createOrReplaceTempView('employee_uni')

spark.sql('''
select unique_id, name from employees left join employee_uni on employees.id = employee_uni.id;
''').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+


In [13]:
spark.stop()